<a href="https://colab.research.google.com/github/dsameendra/project-sentitube-chrome-extension/blob/main/fyp_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers accelerate datasets torch torchvision scikit-learn pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 844.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [7]:
import pandas as pd
import numpy as np
import torch
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset

# 1. Mount Google Drive
if not torch.cuda.is_available():
    print("⚠️ Warning: No GPU detected. Training will be slow. Go to Runtime > Change runtime type > T4 GPU.")
else:
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")

drive.mount('/content/drive')

# 2. Define File Path
# (මෙතන ඔයාගේ ෆයිල් එකේ නම හරිද බලන්න. CSV ද Excel ද කියලා)
dataset_path = '/content/drive/My Drive/mentalhealthnewdataset.xlsx'
print(f"Loading dataset from: {dataset_path}")

try:
    # Load the Excel file
    mental_health_df = pd.read_excel(dataset_path)
    # (ෆයිල් එක CSV නම් 'read_excel' වෙනුවට 'read_csv' දාන්න)

    print(f"Dataset loaded successfully. Total Raw Rows: {len(mental_health_df)}")

except FileNotFoundError:
    print("❌ Error: File not found. Please verify the file path in Google Drive.")
    raise

# 3. Data Cleaning Pipeline (Advanced)
print("Starting Advanced Data Cleaning...")

# Step 3.1: Remove rows where 'Label' is missing
mental_health_df = mental_health_df.dropna(subset=['Label'])

# Step 3.2: Ensure 'Label' column contains only numbers
# (මේකෙන් අර "Level 0: Normal" වගේ අනවශ්‍ය Headers ඔක්කොම අයින් වෙනවා)
mental_health_df['Label'] = pd.to_numeric(mental_health_df['Label'], errors='coerce')
mental_health_df = mental_health_df.dropna(subset=['Label'])

# Step 3.3: Filter for valid classes (0, 1, 2, 3)
mental_health_df['Label'] = mental_health_df['Label'].astype(int)
mental_health_df = mental_health_df[mental_health_df['Label'].isin([0, 1, 2, 3])]

# --- NEW STEPS (මෙන්න මේ ටිකෙන් තමයි Accuracy එක හැදෙන්නේ) ---

# Step 3.4: Remove Duplicates (එකම වාක්‍යය දෙපාරක් තිබුණොත් අයින් කරනවා)
initial_count = len(mental_health_df)
mental_health_df = mental_health_df.drop_duplicates(subset=['Text'])
print(f"Removed {initial_count - len(mental_health_df)} duplicate rows.")

# Step 3.5: Remove Empty Text rows
mental_health_df = mental_health_df.dropna(subset=['Text'])

# Step 3.6: Shuffle Data (හොඳට කවලම් කරනවා)
mental_health_df = mental_health_df.sample(frac=1, random_state=42).reset_index(drop=True)

# --- Final Status Check ---
print("-" * 30)
print(f"✅ Data Cleaning Complete.")
print(f"Final Rows available for training: {len(mental_health_df)}")
print("-" * 30)
print("Class Distribution (Label Balance):")
print(mental_health_df['Label'].value_counts())
print("-" * 30)

# 4. Check Sample Data
print("First 5 rows of cleaned data:")
print(mental_health_df[['Text', 'Label']].head())

✅ GPU Detected: Tesla T4
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading dataset from: /content/drive/My Drive/mentalhealthnewdataset.xlsx
Dataset loaded successfully. Total Raw Rows: 404
Starting Advanced Data Cleaning...
Removed 0 duplicate rows.
------------------------------
✅ Data Cleaning Complete.
Final Rows available for training: 399
------------------------------
Class Distribution (Label Balance):
Label
1    100
3    100
2    100
0     99
Name: count, dtype: int64
------------------------------
First 5 rows of cleaned data:
                                                Text  Label
0           Ada ude pandara nagitta nisa mawa upset.      1
1  Dying is better than living like this, මට තවත්...      3
2          Ada ude tea eka maru, ammage athe rahayi.      0
3  Hithata loku bayak danenawa, anagathaya gana h...      2
4        Ada hawasa set wemu cricket sellam karanna.      0


In [2]:
#meka run karanna epa
import pandas as pd
import numpy as np
import torch
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset

# 1. Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# 2. Define File Path (Update this path if your file name changes)
dataset_path = '/content/drive/My Drive/mentalhealthnewdataset.xlsx'
print(f"Loading dataset from: {dataset_path}")

try:
    # Load the Excel file into a pandas DataFrame
    mental_health_df = pd.read_excel(dataset_path)
    print(f"Dataset loaded successfully. Total Raw Rows: {len(mental_health_df)}")
except FileNotFoundError:
    print("Error: File not found. Please verify the file path in Google Drive.")
    raise

# 3. Data Cleaning Pipeline
print("Starting Data Cleaning Process...")

# Step 3.1: Remove rows where 'Label' is missing
mental_health_df = mental_health_df.dropna(subset=['Label'])

# Step 3.2: Ensure 'Label' column contains only numbers
mental_health_df['Label'] = pd.to_numeric(mental_health_df['Label'], errors='coerce')
mental_health_df = mental_health_df.dropna(subset=['Label'])

# Step 3.3: Filter for valid classes (0: Normal, 1: Mild, 2: Moderate, 3: Severe)
mental_health_df['Label'] = mental_health_df['Label'].astype(int)
mental_health_df = mental_health_df[mental_health_df['Label'].isin([0, 1, 2, 3])]

print(f"Data Cleaning Complete. Rows available for training: {len(mental_health_df)}")
print(f"Unique Classes Found: {mental_health_df['Label'].unique()}")

Mounting Google Drive...
Mounted at /content/drive
Loading dataset from: /content/drive/My Drive/mentalhealthnewdataset.xlsx
Dataset loaded successfully. Total Raw Rows: 404
Starting Data Cleaning Process...
Data Cleaning Complete. Rows available for training: 399
Unique Classes Found: [0 1 2 3]


In [8]:
# 1. Model Configuration
model_checkpoint = "xlm-roberta-base"
print(f"Initializing Model: {model_checkpoint}")

# Initialize Tokenizer (for converting text to numbers) and Model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4)

# 2. Custom Dataset Class
class MentalHealthDataset(Dataset):
    """
    Custom PyTorch Dataset class to handle Singlish text data.
    """
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item_index):
        text = str(self.texts[item_index])
        label = self.labels[item_index]

        # Tokenize the text
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. Split Data into Training (80%) and Testing (20%) sets
text_list = mental_health_df['Text'].tolist()
label_list = mental_health_df['Label'].tolist()

train_texts, test_texts, train_labels, test_labels = train_test_split(
    text_list, label_list, test_size=0.2, random_state=42
)

# Create Dataset objects
train_dataset = MentalHealthDataset(train_texts, train_labels, tokenizer)
test_dataset = MentalHealthDataset(test_texts, test_labels, tokenizer)

print("Model and Datasets are ready.")

Initializing Model: xlm-roberta-base


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

XLMRobertaForSequenceClassification LOAD REPORT from: xlm-roberta-base
Key                         | Status     | 
----------------------------+------------+-
lm_head.bias                | UNEXPECTED | 
lm_head.dense.weight        | UNEXPECTED | 
roberta.pooler.dense.weight | UNEXPECTED | 
lm_head.dense.bias          | UNEXPECTED | 
lm_head.layer_norm.bias     | UNEXPECTED | 
roberta.pooler.dense.bias   | UNEXPECTED | 
lm_head.layer_norm.weight   | UNEXPECTED | 
classifier.dense.bias       | MISSING    | 
classifier.out_proj.weight  | MISSING    | 
classifier.dense.weight     | MISSING    | 
classifier.out_proj.bias    | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Model and Datasets are ready.


In [9]:
# 1. Define Evaluation Metrics
def compute_metrics(prediction_output):
    labels = prediction_output.label_ids
    preds = prediction_output.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {'accuracy': accuracy}

# 2. Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,              # Number of times to iterate over the dataset
    per_device_train_batch_size=4,    # Batch size for training
    per_device_eval_batch_size=4,     # Batch size for evaluation
    eval_strategy="epoch",            # Check accuracy at end of every epoch
    save_strategy="epoch",            # Save model at end of every epoch
    load_best_model_at_end=True,      # Keep the best performing model
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"                  # Disables the W&B prompts
)

# 3. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# 4. Start Training
print("Starting Training Process...")
trainer.train()
print("Training Completed Successfully.")

# 5. Save the Final Model to Google Drive
save_directory = "/content/drive/My Drive/Mental_Health_Project_Final_Model_New"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Model saved successfully to: {save_directory}")

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Starting Training Process...


Epoch,Training Loss,Validation Loss,Accuracy
1,1.444849,1.387245,0.250000
2,1.387447,1.383152,0.325000
3,1.346998,1.555215,0.325000
4,1.210644,1.348637,0.325000
5,1.402750,1.445524,0.287500
6,1.101820,1.279459,0.350000
7,1.305738,1.318113,0.350000
8,1.299468,1.213549,0.450000
9,1.197101,1.295893,0.412500
10,1.247453,1.316142,0.400000


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

Training Completed Successfully.


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Model saved successfully to: /content/drive/My Drive/Mental_Health_Project_Final_Model_New


In [2]:
# Path where the trained model is saved in Google Drive
save_directory = "/content/drive/My Drive/Mental_Health_Project_Final_Model_New"

print("Loading saved model... Please wait.")

# Load the Tokenizer and Model from the saved directory
try:
    tokenizer = AutoTokenizer.from_pretrained(save_directory)
    model = AutoModelForSequenceClassification.from_pretrained(save_directory)

    # Move model to GPU if available (to speed up inference)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print("✅ Model loaded successfully from Google Drive!")
    print(f"Running on device: {device}")

except OSError:
    print("❌ Error: Model path not found. Please check your Google Drive folder.")

Loading saved model... Please wait.


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

✅ Model loaded successfully from Google Drive!
Running on device: cuda


In [10]:
def get_severity_prediction(text_input):
    """
    Predicts the severity level for a single text input.
    Returns: Integer (0-3)
    """
    # Tokenize input
    inputs = tokenizer(text_input, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Move inputs to GPU if available
    device = next(model.parameters()).device
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return predicted_class_id

def analyze_user_risk_framework(user_posts_list):
    """
    Proposed Framework Logic:
    Analyzes a user's timeline (list of posts) and determines the overall risk
    based on the maximum severity detected.
    """
    print(f"\nAnalyzing User Timeline ({len(user_posts_list)} posts detected)...")

    severity_scores = []
    # Mapping integers to readable labels
    labels_map = {0: "Normal", 1: "Mild", 2: "Moderate", 3: "Severe"}

    # Step 1: Analyze each post individually
    for index, post in enumerate(user_posts_list):
        severity_level = get_severity_prediction(post)
        severity_scores.append(severity_level)
        print(f"   Post {index+1}: '{post}' -> Detected Level: {labels_map[severity_level]}")

    # Step 2: Risk Aggregation Logic (Max Risk Approach)
    # If even one post is 'Severe', the user is flagged as High Risk.
    max_severity_level = max(severity_scores)

    print("-" * 60)
    print(f"FINAL SYSTEM ASSESSMENT: {labels_map[max_severity_level]}")

    # Step 3: Clinical Triage Logic
    if max_severity_level == 3:
        print("ALERT: High Risk Detected. Immediate intervention recommended.")
    elif max_severity_level == 2:
        print("WARNING: Moderate Risk. Psychological support suggested.")
    else:
        print("STATUS: Low Risk. No significant immediate indicators.")
    print("-" * 60)

# ==========================================
# SYSTEM DEMONSTRATION
# ==========================================
print("\nSystem Ready for Demonstration.")

# Test Data (You can change these sentences to test)
demo_posts = [

    "mata marenna hithenawa"            # Severe
]

# Execute Framework
analyze_user_risk_framework(demo_posts)


System Ready for Demonstration.

Analyzing User Timeline (1 posts detected)...
   Post 1: 'mata marenna hithenawa' -> Detected Level: Severe
------------------------------------------------------------
FINAL SYSTEM ASSESSMENT: Severe
ALERT: High Risk Detected. Immediate intervention recommended.
------------------------------------------------------------
